# 导入库函数

In [1]:
from sklearn.cluster import KMeans
import torch
from torch.nn import Embedding
from torch.utils.data import DataLoader
import torch.nn as nn
from torch.nn import Parameter
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, average_precision_score
from sklearn import metrics
#from torch_geometric.datasets import Planetoid
EPS = 1e-15

# evaluating indicator

In [2]:
def predict_acc(recons_edges, true_edges):
    predict_graph = recons_edges
    predict_edges = np.array(predict_graph)
    
    ap = average_precision_score(true_edges, predict_edges)
    print("AP： ", ap)
    fpr, tpr, _ = metrics.roc_curve(true_edges, predict_edges, pos_label=1)
    auc = metrics.auc(fpr, tpr)
    print("AUC SCORE: ",auc)
    
    return ap, auc
    #predict_edges[predict_edges>gama] = 1
    #predict_edges[predict_edges<=gama] = 0
    
    #print("ACC： ",accuracy_score(true_edges, predict_edges))
    #print("AP： ",precision_score(true_edges, predict_edges, average='macro'))
    #print("RECALL： ",recall_score(true_edges, predict_edges, average='macro'))
    #print("F1 SCORE： ",f1_score(true_edges, predict_edges, average='macro'))

# model

In [3]:
"""  Given the cluster centroids, update the node representation  """
class reconstruction_graph(nn.Module):
    def __init__(self, NE, alpha=1.0):
        super(reconstruction_graph, self).__init__()
        self.alpha = alpha
        self.nodes_embedding = Parameter(NE)
        
    def forward(self, CC):
        # Calculate the attention score (in nodes) for each node to each cluster
        norm_squared = torch.sum((self.nodes_embedding.unsqueeze(1) - CC)**2, 2)   # Distance from the node to the center of mass
        numerator = 1.0 / (1.0 + (norm_squared / DEC))
        power = float(self.alpha + 1) / 2   
        numerator = numerator**power    
        soft_assignments = (numerator.t() / torch.sum(numerator, 1)).t()       #soft assignment using t-distribution
        
        # Calculate the cluster similarity between nodes (cosine similarity): the larger the value, the more conducive to edge formation
        prod = torch.mm(soft_assignments, soft_assignments.t())       #  numerator
        norm = torch.norm(soft_assignments,p=2,dim=1).unsqueeze(0)      #    denominator 
        clusters_similar = prod.div(torch.mm(norm.t(),norm))
        
        # Calculate the distance between nodes: smaller values favor edge formation
        nodes_distance = torch.norm(self.nodes_embedding[:, None]-self.nodes_embedding, dim=2, p=2)
        nodes_distance = torch.div(nodes_distance, torch.max(nodes_distance))
        
        # The formation probability of the edges is calculated
        distance_similar = torch.div(beta*nodes_distance, clusters_similar)      
        nodes_similar = torch.exp(-distance_similar)
        
        return nodes_similar

class update_nodes_embedding(nn.Module):
    def __init__(self, ne):
        super(update_nodes_embedding, self).__init__()
        NE = ne
        self.reconstruction_module = reconstruction_graph(NE)     
        self.optimizer = torch.optim.SGD(params=self.reconstruction_module.parameters(), lr=LR, momentum=0.9)
        self.loss_function = torch.nn.MSELoss(reduction='sum')
        
    def forward(self, g, CC, edge_train, edge_test):
        self.reconstruction_module.train()
        for epoch in range(5):
            self.optimizer.zero_grad()
            graph_reconstruction = self.reconstruction_module(CC)
            graph_train = torch.take(graph_reconstruction, edge_train)
            loss = self.loss_function(g, graph_train)
            loss.backward()
            self.optimizer.step()
            #print(f'Epoch: {epoch:02d}, Loss: {loss.item():.4f}')
        recons_test_edges = torch.take(graph_reconstruction, edge_test).detach()
        ap, auc = predict_acc(recons_test_edges, test_edge)
        return self.reconstruction_module.nodes_embedding.detach(), ap, auc
    
"""  Type-centroid update aimed at maximizing the prediction effect   """
class ClusteringLayer(nn.Module):
    """  Given the node representation, update the cluster centroids  """
    def __init__(self, cc, alpha=1.0):
        super(ClusteringLayer, self).__init__()
        self.alpha = alpha
        self.cluster_centers = Parameter(cc)
    
    def forward(self, NE):
        norm_squared = torch.sum((NE.unsqueeze(1) - self.cluster_centers)**2, 2) 
        numerator = 1.0 / (1.0 + (norm_squared / DEC))
        power = float(self.alpha + 1) / 2   
        numerator = numerator**power   
        soft_assignments = (numerator.t() / torch.sum(numerator, 1)).t() 
        
        prod = torch.mm(soft_assignments, soft_assignments.t())
        norm = torch.norm(soft_assignments,p=2,dim=1).unsqueeze(0)
        clusters_similar = prod.div(torch.mm(norm.t(),norm))
        
        nodes_distance = torch.norm(NE[:, None]-NE, dim=2, p=2)
        nodes_distance = torch.div(nodes_distance, torch.max(nodes_distance))
        
        distance_similar = torch.div(beta*nodes_distance, clusters_similar)      
        nodes_similar = torch.exp(-distance_similar)
        
        return nodes_similar

def find_cluster_centers(NE, CC, g, edge_train, edge_test, num_epochs=5):        
    clusteringlayer = ClusteringLayer(CC)
    optimizer = torch.optim.SGD(params=clusteringlayer.parameters(), lr=LR, momentum=0.9)
    loss_function = torch.nn.MSELoss(reduction='sum')
    
    for epoch in range(num_epochs):
        optimizer.zero_grad()
        graph_recons = clusteringlayer(NE)
        graph_train = torch.take(graph_recons, edge_train)
        loss = loss_function(g, graph_train)
        loss.backward()
        optimizer.step()
        #print(f'Epoch: {epoch:02d}, Loss: {loss.item():.4f}')     
    recons_test_edges = torch.take(graph_recons, edge_test).detach()
    ap, auc = predict_acc(recons_test_edges, test_edge)
    return clusteringlayer.cluster_centers.detach(), ap, auc

In [4]:
def get_cora_graph(filename_adj, nodes_number):
    edge_index = pd.read_csv(filename_adj, header=None, sep=' ')
    edges = torch.from_numpy(edge_index.values).t().contiguous()
    
    #print("edges number ： ", edges.shape[1])
    
    graph_np = np.zeros((nodes_number, nodes_number))
    for i in range(edges.shape[1]):
        graph_np[edges[0,i], edges[1,i]]=1
        graph_np[edges[1,i], edges[0,i]]=1

    posi_edge = np.argwhere(graph_np == 1)
    posi_edge = posi_edge[posi_edge[:,0]<posi_edge[:,1]]         # Only the upper-left corner matrix is taken
    posi_edge_number = posi_edge.shape[0]
    
    nega_edge = np.argwhere(graph_np == 0) 
    nega_edge = nega_edge[nega_edge[:,0]<nega_edge[:,1]]     # Only the upper-left corner matrix is taken
    
    positive_index = np.random.choice(range(posi_edge_number),int(posi_edge_number*0.9),replace=False)
    choose_positive = posi_edge[positive_index]
    posi_not_choose = np.setdiff1d(range(posi_edge_number), positive_index)
    test_positive = posi_edge[posi_not_choose]
    
    negative_index = np.random.choice(range(nega_edge.shape[0]),int(posi_edge_number*0.9)*6,replace=False)
    choose_negative = nega_edge[negative_index]
    nega_not_choose = np.setdiff1d(range(nega_edge.shape[0]), negative_index)
    test_nega_index = np.random.choice(nega_not_choose,len(posi_not_choose),replace=False)
    test_negative = nega_edge[test_nega_index]
    
    train_posi = [list(choose_positive[i]) for i in range(len(choose_positive))]
    train_nega = [list(choose_negative[i]) for i in range(len(choose_negative))]
    train_index = train_posi + train_nega
    train_mask = [train_index[i][0]*nodes_number+train_index[i][1] for i in range(len(train_index))]
    train_mask = torch.tensor(train_mask)
    
    test_posi = [list(test_positive[i]) for i in range(len(test_positive))]
    test_nega = [list(test_negative[i]) for i in range(len(test_negative))]
    test_index = test_posi + test_nega
    test_mask = [test_index[i][0]*nodes_number+test_index[i][1] for i in range(len(test_index))]
    test_mask = torch.tensor(test_mask)
    
    graph_tensor = torch.from_numpy(graph_np).float()
    train_edge = torch.take(graph_tensor, train_mask)
    test_edge = np.array(torch.take(graph_tensor, test_mask))
    
    return train_edge, test_edge, train_mask, test_mask

In [5]:
filename = 'datasets/Celegans.txt'    # The node index starts at 0, sep=None
nodes_number = 297

In [10]:
embedding_dim = 12
n_clusters = 24
alpha = 1
beta = 4.2
DEC = 5.0
LR = 0.4

all_ap_first_ap = []
all_ap_first_auc = []

all_auc_first_ap = []
all_auc_first_auc = []

for i in range(10):
    train_edge, test_edge, train_mask, test_mask = get_cora_graph(filename, nodes_number)
    ini_embedding = Embedding(nodes_number, embedding_dim, sparse=True)       
    raw_nodes_embedding = ini_embedding.weight.detach()        # Initial node embedding

    kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(raw_nodes_embedding)
    cluster_centers = kmeans.cluster_centers_
    raw_cluster_centers = torch.tensor(cluster_centers, dtype=torch.float)     # Initial cluster centroids
    
    ap_first__ap = 0
    ap_first_auc = 0
    auc_first__ap = 0
    auc_first_auc = 0
    for module_epoch in range(300):
        print("######################### Module cycle ： %d ##########################"%module_epoch)
        update_nodes_module = update_nodes_embedding(raw_nodes_embedding)
        raw_nodes_embedding, nodes_ap, nodes_auc = update_nodes_module(train_edge, raw_cluster_centers, train_mask, test_mask)
        if nodes_ap > ap_first__ap:
            ap_first__ap = nodes_ap
            ap_first_auc = nodes_auc
        if nodes_auc > auc_first_auc:
            auc_first_auc = nodes_auc
            auc_first__ap = nodes_ap
        raw_cluster_centers, cluster_ap, cluster_auc = find_cluster_centers(raw_nodes_embedding, raw_cluster_centers, train_edge, train_mask, test_mask)
        if cluster_ap > ap_first__ap:
            ap_first__ap = cluster_ap
            ap_first_auc = cluster_auc
        if cluster_auc > auc_first_auc:
            auc_first_auc = cluster_auc
            auc_first__ap = cluster_ap
    all_ap_first_ap.append(ap_first__ap)
    all_ap_first_auc.append(ap_first_auc)
    all_auc_first_ap.append(auc_first__ap)
    all_auc_first_auc.append(auc_first_auc)

######################### Module cycle ： 0 ##########################
AP：  0.6596644605135216
AUC SCORE:  0.6426608977825853
AP：  0.6772673078594966
AUC SCORE:  0.6632341806381828
######################### Module cycle ： 1 ##########################
AP：  0.6990174212521072
AUC SCORE:  0.6851487290427257
AP：  0.7018077992216551
AUC SCORE:  0.6880043266630611
######################### Module cycle ： 2 ##########################
AP：  0.7078697025638805
AUC SCORE:  0.6887614926987561
AP：  0.7099266329136127
AUC SCORE:  0.6885667928610059
######################### Module cycle ： 3 ##########################
AP：  0.7112630580000554
AUC SCORE:  0.6896917252568955
AP：  0.7117966364055706
AUC SCORE:  0.6895186587344511
######################### Module cycle ： 4 ##########################
AP：  0.7151664155827449
AUC SCORE:  0.691032990805841
AP：  0.7165517575785325
AUC SCORE:  0.6918550567874527
######################### Module cycle ： 5 ##########################
AP：  0.7187785277385399
AUC SCO

AP：  0.8794487180649362
AUC SCORE:  0.8782693347755544
AP：  0.8795514449990884
AUC SCORE:  0.8788967009194159
######################### Module cycle ： 48 ##########################
AP：  0.8790946401528227
AUC SCORE:  0.8793293672255273
AP：  0.8797327204800789
AUC SCORE:  0.8802595997836669
######################### Module cycle ： 49 ##########################
AP：  0.8767502852797833
AUC SCORE:  0.8803677663601946
AP：  0.8769728357874604
AUC SCORE:  0.880454299621417
######################### Module cycle ： 50 ##########################
AP：  0.8763409141636788
AUC SCORE:  0.8805408328826392
AP：  0.8767970913635541
AUC SCORE:  0.880757166035695
######################### Module cycle ： 51 ##########################
AP：  0.8767560920555584
AUC SCORE:  0.8811032990805842
AP：  0.8764783575836245
AUC SCORE:  0.8811032990805842
######################### Module cycle ： 52 ##########################
AP：  0.875781104346704
AUC SCORE:  0.8814277988101676
AP：  0.876148597220273
AUC SCORE:  0.881427

AP：  0.8726492184106314
AUC SCORE:  0.8797404002163332
######################### Module cycle ： 95 ##########################
AP：  0.8731066086246204
AUC SCORE:  0.8800648999459167
AP：  0.8729657781439588
AUC SCORE:  0.8798918334234721
######################### Module cycle ： 96 ##########################
AP：  0.8731880116150412
AUC SCORE:  0.8801514332071391
AP：  0.8731396255227872
AUC SCORE:  0.88
######################### Module cycle ： 97 ##########################
AP：  0.8733408130828435
AUC SCORE:  0.8801946998377502
AP：  0.8732257341581031
AUC SCORE:  0.8801081665765278
######################### Module cycle ： 98 ##########################
AP：  0.873301021916674
AUC SCORE:  0.8800865332612224
AP：  0.8732684270394269
AUC SCORE:  0.880151433207139
######################### Module cycle ： 99 ##########################
AP：  0.8733424265991796
AUC SCORE:  0.8801946998377501
AP：  0.8733427054723426
AUC SCORE:  0.8801730665224446
######################### Module cycle ： 100 ###########

AP：  0.8762233139026996
AUC SCORE:  0.88322336398053
AP：  0.8762086583565205
AUC SCORE:  0.8831151974040021
######################### Module cycle ： 141 ##########################
AP：  0.8762551578954352
AUC SCORE:  0.8832017306652244
AP：  0.8762423139509237
AUC SCORE:  0.8832017306652244
######################### Module cycle ： 142 ##########################
AP：  0.8762620999956424
AUC SCORE:  0.8832017306652244
AP：  0.8761296496586429
AUC SCORE:  0.8831368307193078
######################### Module cycle ： 143 ##########################
AP：  0.876258013315694
AUC SCORE:  0.8832449972958355
AP：  0.87615090721222
AUC SCORE:  0.8832449972958356
######################### Module cycle ： 144 ##########################
AP：  0.8762715965456584
AUC SCORE:  0.8833315305570577
AP：  0.8762078496861169
AUC SCORE:  0.8833315305570579
######################### Module cycle ： 145 ##########################
AP：  0.8761939369201116
AUC SCORE:  0.8833531638723633
AP：  0.8762640137070892
AUC SCORE:  0.88

AUC SCORE:  0.8852136289886424
######################### Module cycle ： 187 ##########################
AP：  0.8774646783710575
AUC SCORE:  0.8853866955110871
AP：  0.8773832666443333
AUC SCORE:  0.8853001622498647
######################### Module cycle ： 188 ##########################
AP：  0.8774472892203827
AUC SCORE:  0.8853866955110871
AP：  0.8774135508692402
AUC SCORE:  0.8853650621957815
######################### Module cycle ： 189 ##########################
AP：  0.8774910004320925
AUC SCORE:  0.8854515954570038
AP：  0.8773843841237285
AUC SCORE:  0.8853434288804759
######################### Module cycle ： 190 ##########################
AP：  0.8774816120494945
AUC SCORE:  0.8854515954570038
AP：  0.8774288761528676
AUC SCORE:  0.8854083288263928
######################### Module cycle ： 191 ##########################
AP：  0.8774789831981191
AUC SCORE:  0.8854732287723094
AP：  0.8774258115867494
AUC SCORE:  0.8854083288263925
######################### Module cycle ： 192 ##############

...

AP：  0.8696702501789823
AUC SCORE:  0.8760411032990806
AP：  0.8693466168192596
AUC SCORE:  0.8758247701460249
######################### Module cycle ： 82 ##########################
AP：  0.8694713693526624
AUC SCORE:  0.8761276365603028
AP：  0.8695356893569206
AUC SCORE:  0.8761925365062195
######################### Module cycle ： 83 ##########################
AP：  0.869540633868238
AUC SCORE:  0.876279069767442
AP：  0.8701107364487664
AUC SCORE:  0.8764521362898864
######################### Module cycle ： 84 ##########################
AP：  0.8701321336130186
AUC SCORE:  0.8765386695511087
AP：  0.8705693458606967
AUC SCORE:  0.876819902650081
######################### Module cycle ： 85 ##########################
AP：  0.8706134177920264
AUC SCORE:  0.8769497025419145
AP：  0.8705187839693151
AUC SCORE:  0.8769929691725258
######################### Module cycle ： 86 ##########################
AP：  0.8709815959156618
AUC SCORE:  0.8773823688480259
AP：  0.871127306921968
AUC SCORE:  0.877620

AP：  0.8750065465014896
AUC SCORE:  0.8859275283937263
AP：  0.8749976368900778
AUC SCORE:  0.885840995132504
######################### Module cycle ： 129 ##########################
AP：  0.8749771756516507
AUC SCORE:  0.8859491617090319
AP：  0.8751045898713301
AUC SCORE:  0.8859491617090319
######################### Module cycle ： 130 ##########################
AP：  0.8750836716864918
AUC SCORE:  0.8859491617090319
AP：  0.8750959002953617
AUC SCORE:  0.8859491617090318
######################### Module cycle ： 131 ##########################
AP：  0.875217701252841
AUC SCORE:  0.8860573282855597
AP：  0.8751190279621167
AUC SCORE:  0.8859491617090319
######################### Module cycle ： 132 ##########################
AP：  0.8753126494860143
AUC SCORE:  0.8861438615467819
AP：  0.8752394330973041
AUC SCORE:  0.8860356949702543
######################### Module cycle ： 133 ##########################
AP：  0.875279629329566
AUC SCORE:  0.886143861546782
AP：  0.8753923211238777
AUC SCORE:  0.8

AP：  0.883827593213919
AUC SCORE:  0.8899513250405625
AP：  0.883838870328024
AUC SCORE:  0.8898215251487291
######################### Module cycle ： 175 ##########################
AP：  0.885085854413159
AUC SCORE:  0.8904488912925905
AP：  0.8861299619330005
AUC SCORE:  0.8906652244456463
######################### Module cycle ： 176 ##########################
AP：  0.8884102754920967
AUC SCORE:  0.8916603569497026
AP：  0.889771123500465
AUC SCORE:  0.8923309897241752
######################### Module cycle ： 177 ##########################
AP：  0.8906931470451137
AUC SCORE:  0.8929583558680367
AP：  0.8909497683804135
AUC SCORE:  0.8933261222282315
######################### Module cycle ： 178 ##########################
AP：  0.8913250910419407
AUC SCORE:  0.8936506219578151
AP：  0.8914592878616926
AUC SCORE:  0.8938669551108708
######################### Module cycle ： 179 ##########################
AP：  0.891750535477513
AUC SCORE:  0.8941481882098432
AP：  0.8918237250913175
AUC SCORE:  0.89

AUC SCORE:  0.8984964845862629
######################### Module cycle ： 221 ##########################
AP：  0.8976120311290696
AUC SCORE:  0.8985830178474852
AP：  0.8975424414634079
AUC SCORE:  0.8985181179015683
######################### Module cycle ： 222 ##########################
AP：  0.8976850716390027
AUC SCORE:  0.8986695511087074
AP：  0.8975232960975661
AUC SCORE:  0.8984964845862629
######################### Module cycle ： 223 ##########################
AP：  0.8977181881847913
AUC SCORE:  0.8986479177934019
AP：  0.8976532494582913
AUC SCORE:  0.8985830178474852
######################### Module cycle ： 224 ##########################
AP：  0.8976818124711368
AUC SCORE:  0.8986046511627905
AP：  0.8976673880307927
AUC SCORE:  0.8986046511627908
######################### Module cycle ： 225 ##########################
AP：  0.8976753058463849
AUC SCORE:  0.8986262844780963
AP：  0.8976552706538754
AUC SCORE:  0.8986046511627906
######################### Module cycle ： 226 ##############

AP：  0.8983781226019899
AUC SCORE:  0.8986262844780962
AP：  0.8983050574982689
AUC SCORE:  0.8984964845862627
######################### Module cycle ： 268 ##########################
AP：  0.8983472901251143
AUC SCORE:  0.8986046511627906
AP：  0.8982714981245258
AUC SCORE:  0.8984748512709572
######################### Module cycle ： 269 ##########################
AP：  0.8983782894372284
AUC SCORE:  0.8986262844780962
AP：  0.8980302343158506
AUC SCORE:  0.8984315846403461
######################### Module cycle ： 270 ##########################
AP：  0.8982155898180811
AUC SCORE:  0.898691184424013
AP：  0.8980575110230415
AUC SCORE:  0.8984748512709573
######################### Module cycle ： 271 ##########################
AP：  0.8982514571567566
AUC SCORE:  0.8987128177393185
AP：  0.898122956930145
AUC SCORE:  0.8985613845321795
######################### Module cycle ： 272 ##########################
AP：  0.8983046112500782
AUC SCORE:  0.8987777176852353
AP：  0.8981373091697551
AUC SCORE:  0

In [11]:
print("################################  SUM UP #####################################")
print("AP FIRST AP MEAN: ", np.array(all_ap_first_ap).mean())
print("AP FIRST AP STD: ", np.array(all_ap_first_ap).std())

print("AP FIRST AUC MEAN: ", np.array(all_ap_first_auc).mean())
print("AP FIRST AUC STD: ", np.array(all_ap_first_auc).std())

print("AUC FIRST AP MEAN: ", np.array(all_auc_first_ap).mean())
print("AUC FIRST AP STD: ", np.array(all_auc_first_ap).std())

print("AUC FIRST AUC MEAN: ", np.array(all_auc_first_auc).mean())
print("AUC FIRST AUC STD: ", np.array(all_auc_first_auc).std())

################################  SUM UP #####################################
AP FIRST AP MEAN:  0.8785384166068697
AP FIRST AP STD:  0.011031717811491494
AP FIRST AUC MEAN:  0.8882271498107086
AP FIRST AUC STD:  0.006869421086684049
AUC FIRST AP MEAN:  0.8781094412647237
AUC FIRST AP STD:  0.011116990578089719
AUC FIRST AUC MEAN:  0.8892525689561925
AUC FIRST AUC STD:  0.006369129091195055
